# play input video

In [1]:
from IPython.display import HTML
from base64 import b64encode
def play_video(filename):
  html = ''
  video = open(filename,'rb').read()
  src = 'data:video/mp4;base64,' + b64encode(video).decode()
  html += fr'<video width=900 controls autoplay loop><source src="%s" type="video/mp4"></video>' % src
  return HTML(html)

play_video("final_project_speed_estimation/testing_singkat.mp4")

FileNotFoundError: [Errno 2] No such file or directory: 'final_project_speed_estimation/testing_singkat.mp4'

# Import Dependencies

In [ ]:
import cv2
import torch
import time
import colorsys
import numpy as np
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort

In [ ]:
print(torch.__config__.parallel_info())
import os
os.environ['OMP_NUM_THREADS'] = "12"
os.environ['OMP_DISPLAY_ENV'] = "TRUE"
os.environ['OMP_PROC_BIND'] = "TRUE"
os.environ['OMP_SCHEDULE'] = "STATIC"
os.environ['GOMP_CPU_AFFINITY'] = "0-12"

ATen/Parallel:
	at::get_num_threads() : 10
	at::get_num_interop_threads() : 10
OpenMP 2019
	omp_get_max_threads() : 10
Intel(R) oneAPI Math Kernel Library Version 2023.1-Product Build 20230303 for Intel(R) 64 architecture applications
	mkl_get_max_threads() : 10
Intel(R) MKL-DNN v3.5.3 (Git Hash 66f0cb9eb66affd2da3bf5f8d897376f04aae6af)
std::thread::hardware_concurrency() : 12
Environment variables:
	OMP_NUM_THREADS : 1
	MKL_NUM_THREADS : [not set]
ATen parallel backend: OpenMP



In [ ]:
SOURCE_VIDEO_PATH = "dataset/FKH02/FKH02a_part_2.mp4"
OUTPUT_VIDEO_PATH = "out/Amplaz02a_final_part2.mp4"

BLUR_ID = None
CONF = 0.4

In [ ]:
import cv2
import numpy as np

points = []
current_frame_roi_selector = None

def select_roi_points(event, x, y, flags, param):
    global points, current_frame_roi_selector
    if event == cv2.EVENT_LBUTTONDOWN:
        points.append((x, y))
        cv2.circle(current_frame_roi_selector, (x, y), 5, (0, 255, 0), -1)
        if len(points) > 1:
            cv2.line(current_frame_roi_selector, points[-2], points[-1], (255, 0, 0), 2)
        if len(points) == 4: # Misal kita butuh 4 titik
            cv2.line(current_frame_roi_selector, points[-1], points[0], (255,0,0), 2) # Tutup poligon
        # cv2_imshow(current_frame_roi_selector) # Changed from cv2.imshow - Replace with cv2.imshow
        cv2.imshow("Select ROI - Tekan 's' untuk simpan, 'r' untuk reset", current_frame_roi_selector)


cap = cv2.VideoCapture(SOURCE_VIDEO_PATH)
ret, frame_for_roi = cap.read()
cap.release()

if not ret:
    print("Gagal membaca frame dari video.")
    exit()

current_frame_roi_selector = frame_for_roi.copy()

cv2.namedWindow("Select ROI - Tekan 's' untuk simpan, 'r' untuk reset")
cv2.imshow("Select ROI - Tekan 's' untuk simpan, 'r' untuk reset", current_frame_roi_selector) # Changed from cv2_imshow
cv2.setMouseCallback("Select ROI - Tekan 's' untuk simpan, 'r' untuk reset", select_roi_points)

while True:
    key = cv2.waitKey(1) & 0xFF
    if key == ord('s'): # Simpan titik
        if len(points) >= 3: # Minimal 3 titik untuk poligon
            print("Koordinat ROI yang dipilih:")
            print(f"SOURCE_POLYGONE = np.array({points}, dtype=np.int32)")
            break
        else:
            print("Pilih minimal 3 titik.")
    elif key == ord('r'): # Reset titik
        points = []
        current_frame_roi_selector = frame_for_roi.copy()
        cv2.imshow("Select ROI - Tekan 's' untuk simpan, 'r' untuk reset", current_frame_roi_selector)
    elif key == ord('q'): # Keluar tanpa simpan
        break

cv2.destroyAllWindows()

Koordinat ROI yang dipilih:
SOURCE_POLYGONE = np.array([(511, 53), (725, 75), (670, 253), (406, 187)], dtype=np.int32)


In [ ]:
FRAME_WIDTH=500
FRAME_HEIGHT=1000

# fkh01 SOURCE_POLYGONE = np.array([(398, 463), (1470, 474), (1907, 1039), (3, 1049)], dtype=np.int32)
# amplaz01 SOURCE_POLYGONE = np.array([(663, 150), (1234, 154), (1701, 1009), (205, 1014)], dtype=np.int32)
# fkh02
SOURCE_POLYGONE = np.array([(601, 52), (1296, 56), (1809, 690), (58, 669)], dtype=np.int32)
BIRD_EYE_VIEW = np.array([[0, 0], [FRAME_WIDTH, 0], [FRAME_WIDTH, FRAME_HEIGHT],[0, FRAME_HEIGHT]], dtype=np.float32)

# Convert SOURCE_POLYGONE to float32 for perspective transform
M = cv2.getPerspectiveTransform(SOURCE_POLYGONE.astype(np.float32), BIRD_EYE_VIEW)

In [ ]:

cap = cv2.VideoCapture(SOURCE_VIDEO_PATH)
if not cap.isOpened():
    raise RuntimeError("Gagal membuka video")

print("▶️ Tekan 'c' untuk capture frame, 'q' untuk keluar.")
while True:
    ret, frame = cap.read()
    if not ret:
        print("❌ Habis atau gagal baca frame.")
        break

    # Gambar outline ROI
    disp = frame.copy()
    pts = SOURCE_POLYGONE.reshape((-1,1,2)).astype(np.int32)
    cv2.polylines(disp, [pts], True, (0,255,0), 2)
    cv2.putText(disp, "Press 'c' to capture", (10,30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,0), 2)

    cv2.imshow("1. Select Frame", disp)
    k = cv2.waitKey(1) & 0xFF
    if k == ord('c'):
        selected = frame.copy()
        break
    elif k == ord('q'):
        cap.release()
        cv2.destroyAllWindows()
        exit()

cap.release()
cv2.destroyAllWindows()


bird = cv2.warpPerspective(selected, M, (FRAME_WIDTH, FRAME_HEIGHT))

win_name = "2. Measure on Bird's-Eye (click 2 pts, 'r' reset, 'q' quit)"
cv2.namedWindow(win_name, cv2.WINDOW_NORMAL)
cv2.resizeWindow(win_name, 800, 600)  # tampilan awal

points = []
canvas = bird.copy()

def on_mouse(event, x, y, flags, param):
    global points, canvas
    if event == cv2.EVENT_LBUTTONDOWN:
        if len(points) < 2:
            points.append((x, y))
            cv2.circle(canvas, (x, y), 5, (0,255,0), -1)
            if len(points) == 2:
                # gambar garis dan hitung jarak
                cv2.line(canvas, points[0], points[1], (255,0,0), 2)
                dx = points[1][0] - points[0][0]
                dy = points[1][1] - points[0][1]
                dist_px = int(np.hypot(dx, dy))
                print(f"▶ Measured pixel length: {dist_px} px")
    cv2.imshow(win_name, canvas)

cv2.setMouseCallback(win_name, on_mouse)
cv2.imshow(win_name, canvas)

while True:
    key = cv2.waitKey(0) & 0xFF
    if key == ord('r'):
        # reset
        points = []
        canvas = bird.copy()
        cv2.imshow(win_name, canvas)
        print("🔄 Reset points.")
    elif key == ord('q'):
        break

cv2.destroyAllWindows()

▶️ Tekan 'c' untuk capture frame, 'q' untuk keluar.
▶ Measured pixel length: 84 px


In [18]:
pip install -q supervision ultralytics


Note: you may need to restart the kernel to use updated packages.


# Implementation

In [5]:
def read_frames():
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        yield frame


In [ ]:
def calculate_speed(distance, fps):
    return (distance *fps)*3.6

In [ ]:
# note : ganti faktor dengan meter/piksel optimal
def calculate_distance(p1, p2):
    return np.sqrt((p2[0] - p1[0])**2 + (p2[1] - p1[1])**2)*0.0135

In [8]:
def draw_corner_rect(img, bbox, line_length=30, line_thickness=5, rect_thickness=1,
                     rect_color=(255, 0, 255), line_color=(0, 255, 0)):
    x, y, w, h = bbox
    x1, y1 = x + w, y + h

    if rect_thickness != 0:
        cv2.rectangle(img, bbox, rect_color, rect_thickness)

    # Top Left  x, y
    cv2.line(img, (x, y), (x + line_length, y), line_color, line_thickness)
    cv2.line(img, (x, y), (x, y + line_length), line_color, line_thickness)

    # Top Right  x1, y
    cv2.line(img, (x1, y), (x1 - line_length, y), line_color, line_thickness)
    cv2.line(img, (x1, y), (x1, y + line_length), line_color, line_thickness)

    # Bottom Left  x, y1
    cv2.line(img, (x, y1), (x + line_length, y1), line_color, line_thickness)
    cv2.line(img, (x, y1), (x, y1 - line_length), line_color, line_thickness)

    # Bottom Right  x1, y1
    cv2.line(img, (x1, y1), (x1 - line_length, y1), line_color, line_thickness)
    cv2.line(img, (x1, y1), (x1, y1 - line_length), line_color, line_thickness)

    return img

In [ ]:
# initialize the video capture
cap = cv2.VideoCapture(SOURCE_VIDEO_PATH)
frame_generator = read_frames()
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# create mask to filter detections
pts = SOURCE_POLYGONE.astype(np.int32)
pts = pts.reshape((-1, 1, 2))
polygon_mask = np.zeros((frame_height, frame_width), dtype=np.uint8)
cv2.fillPoly(polygon_mask, [pts], 255)

# initialize the video writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
writer = cv2.VideoWriter(OUTPUT_VIDEO_PATH, fourcc, fps, (frame_width, frame_height))

# initialize the tracker
tracker = DeepSort(max_age=50)

# load YOLO model (use a valid model name or path)
model = YOLO("yolov8m.pt")  

classes_path = "dataset/coco.names"
with open(classes_path, "r") as f:
    class_names = f.read().strip().split("\n")
np.random.seed(42)
colors = np.random.randint(0, 255, size=(len(class_names), 3))

TARGET_CLASSES = ["motorbike", "car", "bicycle", "bus", "truck"]
allowed_ids = [ class_names.index(name) for name in TARGET_CLASSES ]

In [ ]:
import time
import cv2
import numpy as np
import pandas as pd
import os
from datetime import datetime

frame_count = 0
start_time = time.time()
prev_positions = {}
speed_accumulator = {}
track_statistics = {}  
completed_tracks = []  

TRACK_TIMEOUT = 30  
track_last_seen = {}  

def calculate_track_statistics(speeds):
    """Hitung statistik kecepatan untuk satu track dengan outlier handling"""
    if len(speeds) == 0:
        return 0, 0, 0, 0
    
    speeds_array = np.array(speeds)
    
    # filter speeds yang masuk akal (0.1 - 150 km/h untuk kendaraan normal)
    filtered_speeds = speeds_array[(speeds_array >= 0.1) & (speeds_array <= 150)]
    
    if len(filtered_speeds) == 0:
        filtered_speeds = speeds_array
    
    v_ave = np.mean(filtered_speeds)
    v_med = np.median(filtered_speeds)
    
    if len(filtered_speeds) >= 10:
        v_min = np.percentile(filtered_speeds, 5)   # Terendah ke-5%
        v_max = np.percentile(filtered_speeds, 95)  # Tertinggi ke-95%
    elif len(filtered_speeds) >= 5:
        v_min = np.percentile(filtered_speeds, 10)  # Terendah ke-10%
        v_max = np.percentile(filtered_speeds, 90)  # Tertinggi ke-90%
    else:
        v_min = np.min(filtered_speeds)
        v_max = np.max(filtered_speeds)
    
    return v_ave, v_med, v_min, v_max

def save_track_data(track_id, class_id, speeds, class_names):
    """Simpan data track yang sudah selesai"""
    if len(speeds) < 5:  
        return
    
    v_ave, v_med, v_min, v_max = calculate_track_statistics(speeds)
    
    track_data = {
        'track_id': track_id,
        'vehicle_class': class_names[class_id] if class_id < len(class_names) else f'class_{class_id}',
        'class_id': class_id,
        'v_ave': round(v_ave, 2),
        'v_med': round(v_med, 2),
        'v_min': round(v_min, 2),
        'v_max': round(v_max, 2),
        'total_detections': len(speeds),
        'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    }
    
    completed_tracks.append(track_data)

while True:
    try:
        frame = next(frame_generator)
    except StopIteration:
        break

    results = model(frame)
    detect = []

    for pred in results:
        for box in pred.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            confidence = box.conf[0]
            label = int(box.cls[0])  # <- class_id didefinisikan di sini

            # — Filter 1: Confidence threshold
            if confidence < CONF:
                continue

            # — Filter 2: Hanya izinkan jika class_id termasuk allowed_ids
            if label not in allowed_ids:
                continue

            # Validasi koordinat sebelum akses polygon_mask
            cx, cy = (x1 + x2) // 2, (y1 + y2) // 2
            if 0 <= cy < polygon_mask.shape[0] and 0 <= cx < polygon_mask.shape[1]:
                if polygon_mask[cy, cx] == 255:
                    detect.append([[x1, y1, x2 - x1, y2 - y1], confidence, label])

    tracks = tracker.update_tracks(detect, frame=frame)

    valid_tracks = []
    current_track_ids = set()  
    
    for track in tracks:
        if not track.is_confirmed():
            continue

        ltrb = track.to_ltrb()
        class_id = track.get_det_class()
        x1, y1, x2, y2 = map(int, ltrb)
        cx, cy = (x1 + x2) // 2, (y1 + y2) // 2

        # apakah koordinat dalam area polygon
        if not (0 <= cy < polygon_mask.shape[0] and 0 <= cx < polygon_mask.shape[1]):
            continue
        if polygon_mask[cy, cx] == 0:
            continue

        valid_tracks.append(track)
        current_track_ids.add(track.track_id)

    # update track_last_seen untuk track yang terlihat
    for track_id in current_track_ids:
        track_last_seen[track_id] = frame_count

    # cek track yang sudah timeout dan simpan datanya
    tracks_to_remove = []
    for track_id, last_frame in track_last_seen.items():
        if frame_count - last_frame > TRACK_TIMEOUT:
            if track_id in speed_accumulator and len(speed_accumulator[track_id]) > 0:
                class_id = track_statistics.get(track_id, {}).get('class_id', 0)
                save_track_data(track_id, class_id, speed_accumulator[track_id], class_names)
            
            tracks_to_remove.append(track_id)

    # hapus track yang sudah timeout dari semua struktur data
    for track_id in tracks_to_remove:
        if track_id in prev_positions:
            del prev_positions[track_id]
        if track_id in speed_accumulator:
            del speed_accumulator[track_id]
        if track_id in track_statistics:
            del track_statistics[track_id]
        if track_id in track_last_seen:
            del track_last_seen[track_id]

    for track in valid_tracks:
        track_id = track.track_id
        ltrb = track.to_ltrb()
        class_id = track.get_det_class()
        x1, y1, x2, y2 = map(int, ltrb)
        cx, cy = (x1 + x2) // 2, (y1 + y2) // 2

        # menyimpan informasi track untuk perhitugnan
        if track_id not in track_statistics:
            track_statistics[track_id] = {
                'class_id': class_id,
                'first_seen': frame_count
            }

        color = colors[class_id]
        B, G, R = map(int, color)
        text = f"{track_id} - {class_names[class_id]}"
        center_pt = np.array([[(cx, cy)]], dtype=np.float32)

        try:
            transformed_pt = cv2.perspectiveTransform(center_pt, M)
        except:
            continue

        # menghitung jarak dan kecepatan
        if track_id in prev_positions:
            prev_position = prev_positions[track_id]
            distance = calculate_distance(prev_position, transformed_pt[0][0])
            speed = calculate_speed(distance, fps)

            if track_id not in speed_accumulator:
                speed_accumulator[track_id] = []

            speed_accumulator[track_id].append(speed)

            # membatasi jumlah data untuk menghindari memory overflow
            if len(speed_accumulator[track_id]) > 200:
                speed_accumulator[track_id].pop(0)

        prev_positions[track_id] = transformed_pt[0][0]

        # menggambar bounding box & teks
        frame = draw_corner_rect(
            frame,
            (x1, y1, x2 - x1, y2 - y1),
            line_length=15,
            line_thickness=3,
            rect_thickness=1,
            rect_color=(B, G, R),
            line_color=(R, G, B),
        )

        cv2.rectangle(frame, (x1 - 1, y1 - 20), (x1 + len(text) * 10, y1), (B, G, R), -1)
        cv2.putText(frame, text, (x1 + 5, y1 - 7), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
        cv2.polylines(frame, [pts], isClosed=True, color=(255, 0, 0), thickness=2)

        # menampilkan statistik kecepatan real-time
        if track_id in speed_accumulator and len(speed_accumulator[track_id]) > 2:
            v_ave, v_med, v_min, v_max = calculate_track_statistics(speed_accumulator[track_id])
            
            # menampilkan kecepatan rata-rata
            speed_text = f"Avg: {v_ave:.0f} km/h"
            cv2.rectangle(frame, (x1 - 1, y1 - 40), (x1 + len(speed_text) * 10, y1 - 20), (0, 0, 255), -1)
            cv2.putText(frame, speed_text, (x1, y1 - 25), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
            
            # menampilkan statistik tambahan (opsional)
            stats_text = f"Med:{v_med:.0f} Min:{v_min:.0f} Max:{v_max:.0f}"
            cv2.rectangle(frame, (x1 - 1, y1 - 60), (x1 + len(stats_text) * 6, y1 - 40), (0, 100, 0), -1)
            cv2.putText(frame, stats_text, (x1, y1 - 45), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (255, 255, 255), 1)

        # menerapkan gaussian blur untuk objek tertentu
        if BLUR_ID is not None and class_id == BLUR_ID:
            if 0 <= x1 < x2 <= frame.shape[1] and 0 <= y1 < y2 <= frame.shape[0]:
                frame[y1:y2, x1:x2] = cv2.GaussianBlur(frame[y1:y2, x1:x2], (99, 99), 3)

    writer.write(frame)
    frame_count += 1

    if frame_count % 10 == 0:
        elapsed_time = time.time() - start_time
        fps_calc = frame_count / elapsed_time
        print(f"FPS: {fps_calc:.2f}, Completed tracks: {len(completed_tracks)}")

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

for track_id, speeds in speed_accumulator.items():
    if len(speeds) > 0:
        class_id = track_statistics.get(track_id, {}).get('class_id', 0)
        save_track_data(track_id, class_id, speeds, class_names)

# menyiapkan dataframe untuk analisis
if len(completed_tracks) > 0:
    df = pd.DataFrame(completed_tracks)
    
    print(f"\n=== DATA COLLECTION SUMMARY ===")
    print(f"Total vehicles tracked: {len(completed_tracks)}")
    print(f"\nVehicle class distribution:")
    print(df['vehicle_class'].value_counts())
    print(f"\nSpeed statistics summary:")
    print(df[['v_ave', 'v_med', 'v_min', 'v_max']].describe())
    print(f"\nSample data (first 5 rows):")
    print(df.head())
    
else:
    print("No vehicle data collected.")
    df = pd.DataFrame()  

cap.release()
writer.release()
cv2.destroyAllWindows()

print(f"\nProcessing completed. Data stored in DataFrame 'df'.")


0: 384x640 4 persons, 5 cars, 3 motorcycles, 485.4ms
Speed: 6.0ms preprocess, 485.4ms inference, 15.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 5 cars, 3 motorcycles, 416.4ms
Speed: 5.2ms preprocess, 416.4ms inference, 2.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 4 cars, 3 motorcycles, 1 truck, 349.5ms
Speed: 2.9ms preprocess, 349.5ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 persons, 5 cars, 4 motorcycles, 344.8ms
Speed: 5.8ms preprocess, 344.8ms inference, 3.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 5 cars, 5 motorcycles, 1 truck, 415.8ms
Speed: 2.2ms preprocess, 415.8ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 persons, 5 cars, 4 motorcycles, 1 truck, 391.2ms
Speed: 2.6ms preprocess, 391.2ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 persons, 4 cars, 4 motorcycles, 1 truck, 372.9

In [32]:
df

,track_id,vehicle_class,class_id,v_ave,v_med,v_min,v_max,total_detections,timestamp
0,2,car,2,23.610001,24.360001,11.970000,31.040001,60,2025-06-02 16:24:10
1,1,motorbike,3,19.170000,20.400000,5.650000,27.340000,74,2025-06-02 16:24:15
2,4,motorbike,3,20.959999,21.340000,11.560000,30.030001,65,2025-06-02 16:24:20
3,6,car,2,23.959999,25.170000,11.280000,28.389999,59,2025-06-02 16:24:35
4,10,motorbike,3,26.360001,26.510000,13.710000,38.040001,49,2025-06-02 16:24:45
...,...,...,...,...,...,...,...,...,...
94,199,car,2,16.080000,16.969999,7.080000,20.070000,65,2025-06-02 16:37:10
95,202,motorbike,3,31.740000,30.809999,20.379999,46.799999,45,2025-06-02 16:37:10
96,203,motorbike,3,21.700001,21.639999,8.360000,32.639999,57,2025-06-02 16:37:10
97,204,car,2,14.930000,17.200001,7.270000,19.379999,30,2025-06-02 16:37:10


In [37]:
df.to_csv("out_csv/Amplaz02a_part2.csv", index=False)